In [1]:
# Import statements
import arcpy
import os
import shutil
import gc
arcpy.env.overwriteOutput = True

This tool uses soil data, landcover data, and watershed outlines to calculate curve numbers and impervious areas for use in infiltration calculations.

Soil data can be downloaded from the web soil survey or SSURGO (https://www.arcgis.com/apps/View/index.html?appid=cdc49bd63ea54dd2977f3f2853e07fff) and landcover data can be obtained from the NLCD.

In [ ]:
# File paths
landcover_full = r'F:\PyForGIS\FinalProject\BismarckSubWS\Full_Buildout_Land_Use.shp'
soils_full = r'F:\PyForGIS\FinalProject\BismarckSubWS\soildata.shp'
subwatersheds = r'F:\PyForGIS\FinalProject\BismarckSubWS\subcatchments.shp'
workspace = r'F:\PyForGIS\FinalProject\BismarckSubWS'
output = r'F:\PyForGIS\FinalProject\BismarckSubWS\subcatchments_ga.shp'

In [ ]:
# Variables
ksat_field = "ksat"
init_sm_field="theta_init"
csuct_filed="cap_suct"

In [ ]:
# Intermediate file names
temp_folder = os.path.join(workspace, "Scratch")
arcpy.env.scratchWorkspace = temp_folder
landcover = os.path.join(temp_folder, "landcover.shp")
soils = os.path.join(temp_folder, "soils.shp")
soils_dissolved = os.path.join(temp_folder, "soilsdissolved.shp")
intersected_catchments = os.path.join(temp_folder, "intws.shp")
intersected_catchments2 = os.path.join(temp_folder, "intwslc.shp")
dissolved_catchments = os.path.join(temp_folder, "dissws.shp")

In [ ]:
# Create scratch folder
try:
    os.mkdir(temp_folder)
except:
    print("Scratch folder exists")

# Clip land cover and soil data

In [ ]:
arcpy.Clip_analysis(in_features=soils_full, clip_features=subwatersheds, out_feature_class=soils)
arcpy.Clip_analysis(in_features=landcover_full, clip_features=subwatersheds, out_feature_class=landcover)

# Add an ID field to subcatchments for joins

In [ ]:
arcpy.AddField_management(in_table=subwatersheds, field_name="WSID", field_type="LONG", field_precision=10)
arcpy.CalculateField_management(in_table=subwatersheds, field="WSID", expression="!FID!", expression_type="PYTHON3")

# Add an area field for each subcatchment

In [ ]:
arcpy.AddGeometryAttributes_management(Input_Features=subwatersheds, Geometry_Properties="AREA", Area_Unit="ACRES")
arcpy.AddField_management(in_table=subwatersheds, field_name="SUB_AREA", field_type="DOUBLE", field_precision=6)
arcpy.CalculateField_management(in_table=subwatersheds, field="SUB_AREA", expression="!POLY_AREA!", expression_type="PYTHON3")

# Dissolve soils

In [ ]:
# For rows with two hydrologic soil groups, keep the most stringent
with arcpy.da.UpdateCursor(in_table=soils, field_names=[hsg_field]) as cursor:
    for row in cursor:
        if len(row[0])>1:
            print(row[0])
            row[0] = str(row[0])[-1]
            print(row[0])
            cursor.updateRow(row)
# Dissolve soils based on HSG
arcpy.Dissolve_management(in_features=soils, out_feature_class=soils_dissolved, dissolve_field=hsg_field, multi_part="MULTI_PART")

# Intersect Soils, Landcover, and Subwatersheds

In [ ]:
arcpy.Intersect_analysis(in_features=[subwatersheds, soils_dissolved], out_feature_class=intersected_catchments)
arcpy.Intersect_analysis(in_features=[subwatersheds, landcover], out_feature_class=intersected_catchments2)

# Calculated weighted soil criteria

In [ ]:
# Add area weighted initial soil moisture content
arcpy.management.AddField(in_table=intersected_catchments, field_name="Init_SM", field_type="DOUBLE", field_precision=4)
# Add area weighted capillary suction head field
arcpy.management.AddField(in_table=intersected_catchments, field_name="C_Suction", field_type="DOUBLE", field_precision=4)
# Add area weighted hydraulic conductivity field
arcpy.management.AddField(in_table=intersected_catchments, field_name="HK", field_type="DOUBLE", field_precision=4)
# Add an area for each intersection
arcpy.AddGeometryAttributes_management(Input_Features=intersected_catchments, Geometry_Properties="AREA", Area_Unit="ACRES")

# Calculate area weighted hydraulic conductivity field
arcpy.CalculateField_management(in_table=intersected_catchments, field="HK", expression="!"+ksat_field+"!*!POLY_AREA!/!SUB_AREA!", expression_type="PYTHON3")
# Calculate area weighted initial soil moisture field
arcpy.CalculateField_management(in_table=intersected_catchments, field="Init_SM", expression="!"+init_sm_field+"!*!POLY_AREA!/!SUB_AREA!", expression_type="PYTHON3")
# Calculate area weighted capillary suction head field
arcpy.CalculateField_management(in_table=intersected_catchments, field="C_Suction", expression="!"+csuct_field+"!*!POLY_AREA!/!SUB_AREA!", expression_type="PYTHON3")

# Dissolve

In [ ]:
arcpy.management.Dissolve(in_features=intersected_catchments, out_feature_class=soils_dissolved, dissolve_field=["WSID"], statistics_fields=[["HK", "SUM"], ["Init_SM", "SUM"], ["C_Suction", "SUM"]], multi_part="MULTI_PART", unsplit_lines="DISSOLVE_LINES")